In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service

In [2]:
import requests
from bs4 import BeautifulSoup
import json

In [3]:
import pandas as pd

In [51]:
SER = Service('/Users/jimmydelano/Downloads/chromedriver')
OP = webdriver.ChromeOptions()
URL = 'https://www.lennar.com/new-homes/alabama/huntsville'

In [52]:
def myHTMLParser(s):
    i = s.find('href=\"')
    newString = s[i+6:]
    i2 = newString.find('\"')
    return newString[:i2]

In [53]:
driver = webdriver.Chrome(service=SER, options=OP)
driver.get(URL)
driver.maximize_window()
time.sleep(10)


try:
    driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/button").click()
except:
    print("No cookies window. Continue...")
    
time.sleep(5)
# try:
#     driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/div/div[3]/div/button[1]').click()
# except:
#     print("Couldn't find list more homes...")


more_homes = True
while more_homes:
    try:
        load_more = driver.find_element(By.XPATH, '//button[normalize-space()="Load more homes"]')
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(5)
    except:
        print("No more homes to load... Scraping text")
        more_homes = False

keep_scraping = True
index = 1
data = []
while keep_scraping:
    if index == 2:
        index += 1
        continue
    
    try:
        result = driver.find_element(By.XPATH, 
                        f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').text.splitlines()
        
        ## data format error with extra column
        if len(result) != 7:
            assert result[0] == 'Flex Cash'
            result = result[1:]
            
        ## correct data format error with rearranging columns
        if result[0] == "Future release":
            result.insert(5, result.pop(1))
        
        
        ## get link
        html = driver.find_element(By.XPATH, 
                        f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').get_attribute('innerHTML')
        url = 'https://www.lennar.com' + myHTMLParser(html)
        result.append(url)
        
        data.append(result)
        index += 1
    except:
        print("Done Scraping info")
        keep_scraping = False

df = pd.DataFrame(data, columns = ["Availability", "Price", "Beds", "Baths", "Sqft", "Address", "Community", "URL"])

No more homes to load... Scraping text
Done Scraping info


In [142]:
# driver.page_source

In [111]:
df = pd.DataFrame(data, columns = ["Availability", "Price", "Beds", "Baths", "Sqft", "Address", "Community", "URL"])

In [112]:
df

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL
0,Move-in ready,"$318,600",3 bd,2 ba,"2,033 ft²","24212 Costello Dr, Athens, AL",Residence 2033 in Browns Crossing West,https://www.lennar.com/new-homes/alabama/hunts...
1,Move-in ready,"$327,900",3 bd,2 ba,"1,911 ft²","14960 Demarie Ln, Athens, AL",Residence 1911 in Browns Crossing West,https://www.lennar.com/new-homes/alabama/hunts...
2,Move-in ready,"$409,950",3 bd,2.5 ba,"2,369 ft²","310 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...
3,Move-in ready,"$409,950",3 bd,2.5 ba,"2,369 ft²","306 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...
4,Move-in ready,"$409,950",3 bd,2.5 ba,"2,369 ft²","308 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...
...,...,...,...,...,...,...,...,...
129,Future release,From $446K,4 bd,3 ba,"3,159 ft²",Residence 3159,Residence 3159 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...
130,Future release,From $418K,4 bd,3 ba,"2,338 ft²",Residence 2338,Residence 2338 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...
131,Future release,From $336K,3 bd,2 ba,"1,846 ft²",Residence 1846,Residence 1846 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...
132,Future release,From $331K,3 bd,2 ba,"1,696 ft²",Residence 1696,Residence 1696 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...


In [113]:
df['Beds'] = df['Beds'].apply(lambda x : float(x.replace(" bd","")))
df['Baths'] = df['Baths'].apply(lambda x : float(x.replace(" ba","")))

In [114]:
df['Sqft'] = df['Sqft'].apply(lambda x : float(x.replace(" ft²", "").replace(",","")))

In [115]:
df['Price'] = df.Price.replace('\D', '', regex=True).astype(int)

In [117]:
df['Price'] = df.apply(lambda row: row['Price'] if row['Availability'] != 'Future release' 
                        else row['Price']*1000, axis=1)

In [118]:
df.head()

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL
0,Move-in ready,318600,3.0,2.0,2033.0,"24212 Costello Dr, Athens, AL",Residence 2033 in Browns Crossing West,https://www.lennar.com/new-homes/alabama/hunts...
1,Move-in ready,327900,3.0,2.0,1911.0,"14960 Demarie Ln, Athens, AL",Residence 1911 in Browns Crossing West,https://www.lennar.com/new-homes/alabama/hunts...
2,Move-in ready,409950,3.0,2.5,2369.0,"310 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...
3,Move-in ready,409950,3.0,2.5,2369.0,"306 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...
4,Move-in ready,409950,3.0,2.5,2369.0,"308 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...


In [120]:
df.to_csv('./huntsville-al.csv', index=False)

In [187]:
def scrapeLocation(url):
    SER = Service('/Users/jimmydelano/Downloads/chromedriver')
    OP = webdriver.ChromeOptions()
    # URL = 'https://www.lennar.com/new-homes/alabama/huntsville'
    driver = webdriver.Chrome(service=SER, options=OP)
    driver.get(url)
    driver.maximize_window()
    time.sleep(10)


    try:
        driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/button").click()
    except:
        print("No cookies window. Continue...")

    time.sleep(5)

    ## Click the load more homes buttons as many times as necessary...
    more_homes = True
    while more_homes:
        try:
            load_more = driver.find_element(By.XPATH, '//button[normalize-space()="Load more homes"]')
            driver.execute_script("arguments[0].click();", load_more)
            time.sleep(5)
        except:
            print("No more homes to load... Scraping text")
            more_homes = False
    
    ## scrape that data
    keep_scraping = True
    index = 1
    data = []
    while keep_scraping:
        if index == 2:
            index += 1
            continue

        try:
            result = driver.find_element(By.XPATH, 
                            f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').text.splitlines()

            ## data format error
            if len(result) != 7:

                ## flex-cash thing -- just drop it
                if result[0] == 'Flex Cash':
                    result = result[1:]

                ## weird thing where sometimes a half bath is a separate column.
                ## grab that and add it to the total baths (divide by 2)
                if 'half ba' in result[4]:
                    i = int(result[4][0]) / 2
                    b = int(result[3][0])
                    result[3] = str(i+b) + " ba"
                    result.pop(4)

                else:
                    raise Exception("Some other data format issue")

            ## correct data format error with rearranging columns
            if result[0] == "Future release":
                result.insert(5, result.pop(1))


            ## get link
            html = driver.find_element(By.XPATH, 
                            f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').get_attribute('innerHTML')
            url = 'https://www.lennar.com' + myHTMLParser(html)
            result.append(url)

            data.append(result)
            index += 1
        except:
            print("Done Scraping info")
            keep_scraping = False

    df = pd.DataFrame(data, columns = ["Availability", "Price", "Beds", "Baths", "Sqft", "Address", "Community", "URL"])
    df['Beds'] = df['Beds'].apply(lambda x : float(x.replace(" bd","")))
    df['Baths'] = df['Baths'].apply(lambda x : float(x.replace(" ba","")))
    df['Sqft'] = df['Sqft'].apply(lambda x : float(x.replace(" ft²", "").replace(",","")))
    df['Price'] = df.Price.replace('\D', '', regex=True).astype(int)
    df['Price'] = df.apply(lambda row: row['Price'] if row['Availability'] != 'Future release' 
                        else row['Price']*1000, axis=1)
    return df
    

In [188]:
df2 = scrapeLocation("https://www.lennar.com/new-homes/alabama/coastal-alabama")

No more homes to load... Scraping text
Done Scraping info


ValueError: Columns must be same length as key

In [191]:
df2

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL
0,Move-in ready,431900,4.0,3.0,2828.0,"1739 Partridge ST, Foley, AL",Residence 2828 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...


In [182]:
SER = Service('/Users/jimmydelano/Downloads/chromedriver')
OP = webdriver.ChromeOptions()
URL = 'https://www.lennar.com/new-homes/alabama/coastal-alabama'
driver = webdriver.Chrome(service=SER, options=OP)
driver.get(URL)
driver.maximize_window()
time.sleep(10)


try:
    driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/button").click()
except:
    print("No cookies window. Continue...")

time.sleep(5)

## Click the load more homes buttons as many times as necessary...
more_homes = True
while more_homes:
    try:
        load_more = driver.find_element(By.XPATH, '//button[normalize-space()="Load more homes"]')
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(5)
    except:
        print("No more homes to load... Scraping text")
        more_homes = False



No more homes to load... Scraping text


In [185]:
## scrape that data
keep_scraping = True
index = 1
data = []
while keep_scraping:
    if index == 2:
        index += 1
        continue
    
    try:
        result = driver.find_element(By.XPATH, 
                        f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').text.splitlines()

        ## data format error
        if len(result) != 7:

            ## flex-cash thing -- just drop it
            if result[0] == 'Flex Cash':
                result = result[1:]

            ## weird thing where sometimes a half bath is a separate column.
            ## grab that and add it to the total baths (divide by 2)
            if 'half ba' in result[4]:
                i = int(result[4][0]) / 2
                b = int(result[3][0])
                result[3] = str(i+b) + " ba"
                result.pop(4)

            else:
                raise Exception("Some other data format issue")

        ## correct data format error with rearranging columns
        if result[0] == "Future release":
            result.insert(5, result.pop(1))


        ## get link
        html = driver.find_element(By.XPATH, 
                        f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').get_attribute('innerHTML')
        url = 'https://www.lennar.com' + myHTMLParser(html)
        result.append(url)

        data.append(result)
        index += 1

    except:
        print("Done Scraping info")
        keep_scraping = False

df = pd.DataFrame(data, columns = ["Availability", "Price", "Beds", "Baths", "Sqft", "Address", "Community", "URL"])
df['Beds'] = df['Beds'].apply(lambda x : float(x.replace(" bd","")))
df['Baths'] = df['Baths'].apply(lambda x : float(x.replace(" ba","")))
df['Sqft'] = df['Sqft'].apply(lambda x : float(x.replace(" ft²", "").replace(",","")))
df['Price'] = df.Price.replace('\D', '', regex=True).astype(int)
df['Price'] = df.apply(lambda row: row['Price'] if row['Availability'] != 'Future release' 
                    else row['Price']*1000, axis=1)

Done Scraping info


ValueError: Columns must be same length as key

In [ ]:
df